In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Colab Projects/MH_DSCRP_PRED_ANLYTCS/survey.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
NLV = df.isnull().sum()
NLV

In [ ]:
df.info()

In [ ]:
Column_list = df.columns.to_list()
Column_list

In [ ]:
NLVP = (NLV/len(df)) * 100
NLVP.sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Values Heatmap')
plt.show()

In [ ]:
import pandas as pd

# Assuming you already have df loaded
df['Treatment_Flag'] = df['treatment'].apply(lambda x: 1 if x == 'Yes' else 0)


# Select only numeric columns for correlation
numeric_df = df.select_dtypes(include=['int64', 'float64'])
print("✅ Numeric columns considered for correlation:")
print(numeric_df.columns.tolist())



corr_matrix = numeric_df.corr()
print("✅ Correlation matrix:")
print(corr_matrix)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlation Heatmap of Numeric Variables")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ✅ Start with your cleaned dataset
# If not already loaded:
# df = pd.read_csv('/content/drive/MyDrive/survey_cleaned.csv')

# ✅ 1. Make a copy of df to work on
df_encoded = df.copy()

# ✅ 2. Encode all categorical columns as numeric codes
for col in df_encoded.columns:
    if df_encoded[col].dtype == 'object':
        df_encoded[col] = df_encoded[col].astype('category').cat.codes
        # cat.codes converts each unique category to a number (0,1,2,...)

# ✅ 3. Compute correlation on the encoded DataFrame
corr_matrix = df_encoded.corr()

# ✅ 4. Plot the correlation heatmap
plt.figure(figsize=(18,16))  # adjust size for readability
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', cbar=True)
plt.title("Correlation Heatmap of All Variables (Encoded)", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
#Since corporate consists of age of people ranging from 18 to 65, we are priorizing that age
df = df[(df['Age'] >= 18) & (df['Age'] <= 65)]

In [ ]:
df.shape

In [ ]:
# Normalize Gender
df['Gender'] = df['Gender'].str.lower().str.strip()
df['Gender'] = df['Gender'].replace({
    'male':'Male','m':'Male','female':'Female','f':'Female',
    'man':'Male','woman':'Female'
})

# All other values as 'Other'
df.loc[~df['Gender'].isin(['Male','Female']), 'Gender'] = 'Other'

In [ ]:
# Fill NAs in key categorical columns with 'Not specified'
for col in ['state','self_employed','family_history','work_interfere','no_employees',
            'remote_work','tech_company','benefits','care_options','wellness_program',
            'seek_help','anonymity','leave','mental_health_consequence',
            'phys_health_consequence','coworkers','supervisor',
            'mental_health_interview','phys_health_interview',
            'mental_vs_physical','obs_consequence']:
    df[col] = df[col].fillna('Not specified')

In [ ]:
df.isnull().sum().sort_values(ascending=False)


In [ ]:
print(df['treatment'].value_counts())
print(df['treatment'].value_counts(normalize=True)*100)


In [ ]:
treatment_by_gender = df.groupby('Gender')['treatment'].value_counts(normalize=True).unstack() * 100
print(treatment_by_gender)

In [ ]:
treatment_by_family = df.groupby('family_history')['treatment'].value_counts(normalize=True).unstack() * 100
print(treatment_by_family)


In [ ]:
df.head()

In [ ]:
df.to_csv('/content/drive/MyDrive/survey_cleaned.csv', index=False)
print("✅ Cleaned dataset saved to Google Drive!")

In [ ]:
# 1. Import sqlite3 (already comes with Python)
import sqlite3
import pandas as pd

# 2. Create a database in memory (temporary) or on disk (permanent)
# For now, let's do in-memory:
conn = sqlite3.connect(':memory:')  # this creates a temporary DB
print("✅ SQLite database created successfully")


In [ ]:
# Load your cleaned CSV into a pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/Colab Projects/MH_DSCRP_PRED_ANLYTCS/survey_cleaned.csv')

df.head()

In [ ]:
# ------------------------------------------
# 2. Push the DataFrame into the SQLite database
# ------------------------------------------
# conn is the database connection we created earlier
# 'survey' is the name we give to the new SQL table
# if_exists='replace' means: if there is already a table named survey, overwrite it
# index=False means: don't add the pandas index as an extra column
df.to_sql('survey', conn, index=False, if_exists='replace')

print("✅ Table 'survey' created in SQLite with", len(df), "rows.")


In [ ]:
# ------------------------------------------
# 3. Verify the table exists in SQLite
# ------------------------------------------
# We ask SQLite: show me all table names in this database
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("✅ Tables currently in this database:")
print(tables)

In [ ]:
# ------------------------------------------
# Count how many rows are in the survey table
# ------------------------------------------
query = "SELECT COUNT(*) AS total_records FROM survey;"

result = pd.read_sql(query, conn)
print("✅ Total rows in the table:")
print(result)


In [ ]:
# ------------------------------------------
# Fetch first 5 rows from the table using SQL
# ------------------------------------------
query = "SELECT * FROM survey LIMIT 5;"

result = pd.read_sql(query, conn)
print("✅ First 5 rows from SQL:")
print(result)

In [ ]:
# ------------------------------------------
# Count how many said Yes or No to treatment
# ------------------------------------------
query = """
SELECT treatment,
       COUNT(*) AS count,
       ROUND(CAST(COUNT(*) AS FLOAT) * 100 / (SELECT COUNT(*) FROM survey), 2) AS percentage
FROM survey
GROUP BY treatment;
"""

result = pd.read_sql(query, conn)
print("✅ Treatment counts and percentages:")
print(result)


In [ ]:
# ------------------------------------------
# Treatment rate grouped by Gender
# ------------------------------------------
query = """
SELECT Gender,
       COUNT(*) AS total_count,
       SUM(CASE WHEN treatment='Yes' THEN 1 ELSE 0 END) AS treated_count,
       ROUND(SUM(CASE WHEN treatment='Yes' THEN 1 ELSE 0 END)*100.0/COUNT(*), 2) AS treatment_rate
FROM survey
GROUP BY Gender;
"""

result = pd.read_sql(query, conn)
print("✅ Treatment rate by Gender:")
print(result)


In [ ]:
# ------------------------------------------
# Top 10 countries by treatment rate
# ------------------------------------------
query = """
SELECT Country,
       COUNT(*) AS total_count,
       SUM(CASE WHEN treatment='Yes' THEN 1 ELSE 0 END) AS treated_count,
       ROUND(SUM(CASE WHEN treatment='Yes' THEN 1 ELSE 0 END)*100.0/COUNT(*), 2) AS treatment_rate
FROM survey
GROUP BY Country
ORDER BY treatment_rate DESC
LIMIT 10;
"""

result = pd.read_sql(query, conn)
print("✅ Top 10 countries by treatment rate:")
print(result)


In [ ]:
# ------------------------------------------
# Treatment rate grouped by family history
# ------------------------------------------
query = """
SELECT family_history,
       COUNT(*) AS total_count,
       SUM(CASE WHEN treatment='Yes' THEN 1 ELSE 0 END) AS treated_count,
       ROUND(SUM(CASE WHEN treatment='Yes' THEN 1 ELSE 0 END)*100.0/COUNT(*), 2) AS treatment_rate
FROM survey
GROUP BY family_history;
"""

result = pd.read_sql(query, conn)
print("✅ Treatment rate by Family History:")
print(result)
